<center>

# Web Intelligence

# Who will win the Tennis Australian Open 2020?

## 03 Building The Prediction Model

### Riccardo Spolaor (864877)

</center>

Modifico il dataframe in modo da poter ottenere un modello adatto alla predizione:
* Rinomino ***Winner*** in ***PlayerA*** e ***Loser*** in ***PlayerB***;
* Rinomino tutte le colonne con informazioni legate al vincitore e al perdente in modo da riflettere il legame ai nuovi nomi assegnati;
* Creo la colonna ***Winner*** che assume i valori:
    * 0 se ha vinto ***PlayerA***;
    * 1 se ha vinto ***PlayerB***;
* Assegno a ***Winner*** tutti valori 0;
* Duplico il dataframe e faccio lo swapping tra tutte le colonne con informazioni sul Giocatore A e quelle con informazioni sul Giocatore B;
* Assegno il valore 1 a ogni entry della colonna ***Winner*** del nuovo dataframe;
* Unisco i due dataframe.

In [38]:
import pandas as pd

In [39]:
dataframe = pd.read_csv('./datasets/dataframe05.csv', low_memory = False)

In [40]:
dataframe.columns.values

array(['csvID', 'ATP', 'Location', 'Tournament', 'Date', 'Series',
       'Court', 'Round', 'Best of', 'Winner', 'Loser', 'PlayedA',
       'PlayedB', 'WinRatioA', 'WinRatioB', 'WRank', 'WRankAdded',
       'LRank', 'LRankAdded', 'W1', 'L1', 'W2', 'L2', 'W3', 'L3', 'W4',
       'L4', 'W5', 'L5', 'Wsets', 'Lsets', 'FadigueTournGamesA',
       'FadigueTournGamesB', 'FadigueTournSetsA', 'FadigueTournSetsB',
       'WPts', 'WPtsAdded', 'LPts', 'LPtsAdded', 'MaxW', 'MaxWAdded',
       'MaxL', 'PlayedCourtA', 'PlayedCourtB', 'WonRatioCourtA',
       'WonRatioCourtB', 'MaxLAdded', 'AvgW', 'AvgWAdded', 'AvgL',
       'AvgLAdded', 'Carpet', 'Clay', 'Grass', 'Hard', 'Awarded',
       'Completed', 'Disqualified', 'Retired', 'Sched', 'Walkover',
       'Federer R.A', 'Federer R.B', 'Nadal R.A', 'Nadal R.B',
       'Djokovic N.A', 'Djokovic N.B', 'Murray A.A', 'Murray A.B',
       'Roddick A.A', 'Roddick A.B', 'Ferrer D.A', 'Ferrer D.B',
       'Berdych T.A', 'Berdych T.B', 'Hewitt L.A', 'Hewitt L.

In [41]:
def renameColumns():
    dataframe.rename(columns={'Winner':'PlayerA', 
                          'Loser':'PlayerB',
                          'WRank':'RankA',
                          'WRankAdded':'RankFilledA',
                          'LRank':'RankB',
                          'LRankAdded':'RankFilledB',
                          'W1':'1A',
                          'L1':'1B',
                          'W2':'2A',
                          'L2':'2B',
                          'W3':'3A',
                          'L3':'3B',
                          'W4':'4A',
                          'L4':'4B',
                          'W5':'5A',
                          'L5':'5B',
                          'Wsets':'setsA',
                          'Lsets':'setsB',
                          'WPts':'PtsA',
                          'LPts':'PtsB',
                          'WPtsAdded':'PtsFilledA',
                          'LPtsAdded':'PtsFilledB',
                          'MaxW':'MaxA',
                          'MaxWAdded':'MaxFilledA',
                          'MaxL':'MaxB',
                          'MaxLAdded':'MaxFilledB',
                          'AvgW':'AvgA',
                          'AvgWAdded':'AvgFilledA',
                          'AvgL':'AvgB',
                          'AvgLAdded':'AvgFilledB',
                          'MinW':'MinA',
                          'MinL':'MinB',
                         }, inplace = True)
    dataframe.insert(0, column ='Winner', value = 0)

In [42]:
renameColumns()
dataframe.columns

Index(['Winner', 'csvID', 'ATP', 'Location', 'Tournament', 'Date', 'Series',
       'Court', 'Round', 'Best of',
       ...
       '10_3MeanA', '10_4MeanA', '10_5MeanA', '10_setsMeanA', '10_1MeanB',
       '10_2MeanB', '10_3MeanB', '10_4MeanB', '10_5MeanB', '10_setsMeanB'],
      dtype='object', length=137)

In [43]:
def getInvertedDataFrame(colToInvert):
        
    players = dataframe['PlayerA'].value_counts().subtract(
        dataframe['PlayerB'].value_counts(), fill_value = 0).sort_values(ascending = False).index.tolist()[:25]
    
    colToInvert += players
    
    dataframeInverted = dataframe.copy(deep = True)

    dataframeInverted['Winner'] = 1
    
    for f in colToInvert:
        dataframeInverted[[f+'A', f+'B']] = dataframeInverted[[f+'B', f+'A']]
        
    dataframeInverted['OpponentsWRatioA>OpponentsWRatioB'] = dataframeInverted.apply(lambda x : int(x['OpponentsWRatioA'] > x['OpponentsWRatioB']), axis = 1)
    dataframeInverted['RankA>RankB'] = dataframeInverted.apply(lambda x : int(x['RankA'] > x['RankB']), axis = 1)
    dataframeInverted['AvgA>AvgB'] = dataframeInverted.apply(lambda x : int(x['AvgA'] > x['AvgB']), axis = 1)
    dataframeInverted['MaxA>MaxB'] = dataframeInverted.apply(lambda x : int(x['MaxA'] > x['MaxB']), axis = 1)
    dataframeInverted['WinRatioA>WinRatioB'] = dataframeInverted.apply(lambda x : int(x['WinRatioA'] > x['WinRatioB']), axis = 1)
    dataframeInverted['WonRatioCourtA>WonRatioCourtB'] = dataframeInverted.apply(lambda x : int(x['WonRatioCourtA'] > x['WonRatioCourtB']), axis = 1)
    
    dataframeInverted['FadigueTournGamesA>FadigueTournGamesB'] = dataframeInverted.apply(lambda x : int(x['FadigueTournGamesA'] > x['FadigueTournGamesB']), axis = 1)
    dataframeInverted['FadigueTournSetsA>FadigueTournSetsB'] = dataframeInverted.apply(lambda x : int(x['FadigueTournSetsA'] > x['FadigueTournSetsB']), axis = 1)
    
    return dataframeInverted

In [44]:
dataframeInverted = getInvertedDataFrame(['Player', 'Rank', 'RankFilled', 'Pts', 'PtsFilled', '1', 
                   '2', '3', '4', '5', 'sets', 'Max', 'Avg', 'Other', 'MaxFilled', 'AvgFilled',  'Played',
       'WinRatio', 'WonRatioCourt', 'PlayedCourt', 'OpponentsWRatio', '10_1Mean', '10_2Mean',
                                          '10_3Mean', '10_4Mean',
               '10_5Mean','10_setsMean', 'FadigueTournGames', 'FadigueTournSets'])



In [45]:
dataframeInverted.head()

,Winner,csvID,ATP,Location,Tournament,Date,Series,Court,Round,Best of,...,10_3MeanA,10_4MeanA,10_5MeanA,10_setsMeanA,10_1MeanB,10_2MeanB,10_3MeanB,10_4MeanB,10_5MeanB,10_setsMeanB
0,1,0.0,1.0,2.0,0.0,2001-01-01,0.0,1.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,1.0,2.0,0.0,2001-01-01,0.0,1.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,0.0,1.0,2.0,0.0,2001-01-01,0.0,1.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,0.0,1.0,2.0,0.0,2001-01-01,0.0,1.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,0.0,1.0,2.0,0.0,2001-01-01,0.0,1.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
with pd.option_context('display.max_columns',None):
    display(dataframeInverted[dataframeInverted['PlayerA'] == 'Nadal R.'].tail(2))
    display(dataframe[dataframe['PlayerB'] == 'Nadal R.'].tail(2))
    
#gamesWinRatioTourn pare avere problemi

,Winner,csvID,ATP,Location,Tournament,Date,Series,Court,Round,Best of,PlayerA,PlayerB,PlayedA,PlayedB,WinRatioA,WinRatioB,RankA,RankFilledA,RankB,RankFilledB,1A,1B,2A,2B,3A,3B,4A,4B,5A,5B,setsA,setsB,FadigueTournGamesA,FadigueTournGamesB,FadigueTournSetsA,FadigueTournSetsB,PtsA,PtsFilledA,PtsB,PtsFilledB,MaxA,MaxFilledA,MaxB,PlayedCourtA,PlayedCourtB,WonRatioCourtA,WonRatioCourtB,MaxFilledB,AvgA,AvgFilledA,AvgB,AvgFilledB,Carpet,Clay,Grass,Hard,Awarded,Completed,Disqualified,Retired,Sched,Walkover,Federer R.A,Federer R.B,Nadal R.A,Nadal R.B,Djokovic N.A,Djokovic N.B,Murray A.A,Murray A.B,Roddick A.A,Roddick A.B,Ferrer D.A,Ferrer D.B,Berdych T.A,Berdych T.B,Hewitt L.A,Hewitt L.B,Del Potro J.M.A,Del Potro J.M.B,Tsonga J.W.A,Tsonga J.W.B,Cilic M.A,Cilic M.B,Gasquet R.A,Gasquet R.B,Wawrinka S.A,Wawrinka S.B,Nishikori K.A,Nishikori K.B,Monfils G.A,Monfils G.B,Ferrero J.C.A,Ferrero J.C.B,Robredo T.A,Robredo T.B,Nalbandian D.A,Nalbandian D.B,Raonic M.A,Raonic M.B,Agassi A.A,Agassi A.B,Haas T.A,Haas T.B,Isner J.A,Isner J.B,Moya C.A,Moya C.B,Davydenko N.A,Davydenko N.B,Gonzalez F.A,Gonzalez F.B,OtherA,OtherB,RankA>RankB,MaxA>MaxB,AvgA>AvgB,OpponentsPlayed,OpponentsWRatioA,OpponentsWRatioB,OpponentsWRatioA>OpponentsWRatioB,FadigueTournGamesA>FadigueTournGamesB,FadigueTournSetsA>FadigueTournSetsB,WonRatioCourtA>WonRatioCourtB,WinRatioA>WinRatioB,10_1MeanA,10_2MeanA,10_3MeanA,10_4MeanA,10_5MeanA,10_setsMeanA,10_1MeanB,10_2MeanB,10_3MeanB,10_4MeanB,10_5MeanB,10_setsMeanB
50962,1,18.0,39.0,49.0,25.0,2019-07-12,4.0,1.0,6.0,5.0,Nadal R.,Federer R.,1123.0,1339.0,0.826358,0.843167,2.0,0.0,3.0,0.0,6.0,7.0,6.0,1.0,3.0,6.0,4.0,6.0,0.0,0.0,1.0,3.0,146.0,153.0,16.0,17.0,7945.0,0.0,6620.0,0.0,1.77,0.0,2.30,90.0,199.0,0.788889,0.899497,0.0,1.71,0.0,2.18,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,40.0,0.6,0.4,1,0,0,0,0,6.1,5.6,5.9,1.9,0.0,3.0,5.9,5.7,4.4,1.2,0.0,2.5
51928,1,18.0,65.0,76.0,19.0,2019-11-02,2.0,0.0,6.0,3.0,Nadal R.,Shapovalov D.,1139.0,138.0,0.827919,0.550725,2.0,0.0,28.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,62.0,75.0,6.0,8.0,9225.0,0.0,1460.0,0.0,1.21,0.0,5.63,588.0,106.0,0.768707,0.603774,0.0,1.18,0.0,4.94,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0,0,2.0,0.5,0.5,0,0,0,1,1,5.8,5.2,3.5,1.0,0.6,2.4,4.8,5.0,1.2,0.0,0.0,1.4


,Winner,csvID,ATP,Location,Tournament,Date,Series,Court,Round,Best of,PlayerA,PlayerB,PlayedA,PlayedB,WinRatioA,WinRatioB,RankA,RankFilledA,RankB,RankFilledB,1A,1B,2A,2B,3A,3B,4A,4B,5A,5B,setsA,setsB,FadigueTournGamesA,FadigueTournGamesB,FadigueTournSetsA,FadigueTournSetsB,PtsA,PtsFilledA,PtsB,PtsFilledB,MaxA,MaxFilledA,MaxB,PlayedCourtA,PlayedCourtB,WonRatioCourtA,WonRatioCourtB,MaxFilledB,AvgA,AvgFilledA,AvgB,AvgFilledB,Carpet,Clay,Grass,Hard,Awarded,Completed,Disqualified,Retired,Sched,Walkover,Federer R.A,Federer R.B,Nadal R.A,Nadal R.B,Djokovic N.A,Djokovic N.B,Murray A.A,Murray A.B,Roddick A.A,Roddick A.B,Ferrer D.A,Ferrer D.B,Berdych T.A,Berdych T.B,Hewitt L.A,Hewitt L.B,Del Potro J.M.A,Del Potro J.M.B,Tsonga J.W.A,Tsonga J.W.B,Cilic M.A,Cilic M.B,Gasquet R.A,Gasquet R.B,Wawrinka S.A,Wawrinka S.B,Nishikori K.A,Nishikori K.B,Monfils G.A,Monfils G.B,Ferrero J.C.A,Ferrero J.C.B,Robredo T.A,Robredo T.B,Nalbandian D.A,Nalbandian D.B,Raonic M.A,Raonic M.B,Agassi A.A,Agassi A.B,Haas T.A,Haas T.B,Isner J.A,Isner J.B,Moya C.A,Moya C.B,Davydenko N.A,Davydenko N.B,Gonzalez F.A,Gonzalez F.B,OtherA,OtherB,RankA>RankB,MaxA>MaxB,AvgA>AvgB,OpponentsPlayed,OpponentsWRatioA,OpponentsWRatioB,OpponentsWRatioA>OpponentsWRatioB,FadigueTournGamesA>FadigueTournGamesB,FadigueTournSetsA>FadigueTournSetsB,WonRatioCourtA>WonRatioCourtB,WinRatioA>WinRatioB,10_1MeanA,10_2MeanA,10_3MeanA,10_4MeanA,10_5MeanA,10_setsMeanA,10_1MeanB,10_2MeanB,10_3MeanB,10_4MeanB,10_5MeanB,10_setsMeanB
50962,0,18.0,39.0,49.0,25.0,2019-07-12,4.0,1.0,6.0,5.0,Federer R.,Nadal R.,1339.0,1123.0,0.843167,0.826358,3.0,0.0,2.0,0.0,7.0,6.0,1.0,6.0,6.0,3.0,6.0,4.0,0.0,0.0,3.0,1.0,153.0,146.0,17.0,16.0,6620.0,0.0,7945.0,0.0,2.30,0.0,1.77,199.0,90.0,0.899497,0.788889,0.0,2.18,0.0,1.71,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,40.0,0.4,0.6,0.0,1.0,1.0,1.0,1.0,5.9,5.7,4.4,1.2,0.0,2.5,6.1,5.6,5.9,1.9,0.0,3.0
51928,0,18.0,65.0,76.0,19.0,2019-11-02,2.0,0.0,6.0,3.0,Shapovalov D.,Nadal R.,138.0,1139.0,0.550725,0.827919,28.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,75.0,62.0,8.0,6.0,1460.0,0.0,9225.0,0.0,5.63,0.0,1.21,106.0,588.0,0.603774,0.768707,0.0,4.94,0.0,1.18,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,2.0,0.5,0.5,0.0,1.0,1.0,0.0,0.0,4.8,5.0,1.2,0.0,0.0,1.4,5.8,5.2,3.5,1.0,0.6,2.4


In [47]:
dataframe = dataframe.append(dataframeInverted, sort = False, ignore_index = True)

In [48]:
def sortMatchesByDate ():
    dataframe['Index'] = dataframe.index
    dataframe.sort_values(['csvID','ATP','Index'],inplace = True)
    dataframe.drop('Index', axis=1, inplace = True)
    
sortMatchesByDate()

In [49]:

dataframe.drop(['Date', '1A', '1B', '2A', '2B', '3A', '3B', '4A', 
                '4B', '5A', '5B', 'setsA', 'setsB', 'Awarded', 'Completed',
       'Disqualified', 'Retired', 'Sched', 'Walkover'], axis = 1, inplace = True)

In [50]:

#PlayedA e B inutili

In [51]:
dataframe = dataframe.reset_index(drop = True)
dataframe.head(50)

,Winner,csvID,ATP,Location,Tournament,Series,Court,Round,Best of,PlayerA,...,10_3MeanA,10_4MeanA,10_5MeanA,10_setsMeanA,10_1MeanB,10_2MeanB,10_3MeanB,10_4MeanB,10_5MeanB,10_setsMeanB
0,0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,3.0,Clement A.,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
1,0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,3.0,Goldstein P.,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
2,0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,3.0,Haas T.,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
3,0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,3.0,Henman T.,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
4,0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,3.0,Hewitt L.,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
5,0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,3.0,Johansson T.,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
6,0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,3.0,Ljubicic I.,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
7,0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,3.0,Malisse X.,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
8,0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,3.0,Martin A.,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
9,0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,3.0,Massu N.,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0


In [52]:
dataframe.to_csv('./datasets/dataframe0.csv', index = False)